![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   





# import

In [134]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

#Модуль для определения тональности текста
from textblob import TextBlob

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [135]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [136]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [137]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [138]:
df_train.info()

In [139]:
df_train.head(5)

In [140]:
df_test.info()

In [141]:
df_test.head(5)

In [142]:
sample_submission.head(5)

In [143]:
sample_submission.info()

In [144]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [145]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [146]:
data.sample(5)

In [147]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

## Функции предобработки

In [148]:
#Преоборазование строки в двумерный список
def string_to_list(string):
    open_char = "["
    close_char = "]"
    offset = 1
    final_list = []
    
    while string.find(open_char,offset) > -1:
        close_pos = string.find(close_char,offset)
        sublist = string[offset+1:close_pos].split("', '")
        
        #Убираем лишнюю скобку
        for index,val in enumerate(sublist): 
            sublist[index] = val.replace("'","")
    
        offset = string.find(open_char,close_pos)
        
        final_list.append(sublist)
    
    return final_list

#Определение тональности текста. Вход - pd.Series. Возвращает массив (n,1,[p,s]) - n - число строк входящего reviews, p - полярность текста, s- субъективная оценка
def get_text_score(reviews):
    #final_scores - возвразщаемый массив
    final_scores = []
    #проходим по строкам
    for row in reviews:
        #curr_score - скоринг для текущей строки
        curr_score = list([0,0])
        for i in range(len(row[0])):
            if row[0][i] != "":
                curr_score = [x+y for x, y in zip(curr_score, TextBlob(row[0][i]).sentiment)]
                
        final_scores.append([curr_score])

    return np.array(final_scores)

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

### 1.1. Признак Number_of_Reviews

In [149]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [150]:
data['Number_of_Reviews_isNAN']

In [151]:
data[data['Number of Reviews'].isna()]

In [152]:
# Заполняем пропуск по отзывам медианным значением отзывом для города, в котором расположен ресторан
cities_name = data['City'].unique()
#Получаем словарь индекс - город, значение - среднее количество отзывов по городу.
num_rev_per_city = dict()
for city in cities_name:
    num_rev_per_city[city] = data[data.City == city]['Number of Reviews'].median()

data['Number of Reviews'] = data.apply(lambda row: num_rev_per_city[row.City] if np.isnan(row['Number of Reviews']) else row['Number of Reviews'], axis = 1)


### 1.2. Признак Reviews

In [153]:
#Заменим NaN на [[],[]]. Все остальные пустые значения в признаке Reviews обозначаются именно так. Позже их будет проще обработать в таком формате.
data.loc[data.Reviews.isna(),'Reviews'] = '[[],[]]'

### 1.3. Признак Price Range

Добавим признак, отмечающий наличие пропуска

In [154]:
data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')

In [155]:
data['Price Range'].value_counts()

Наиболее часто встречающийся вариант - \\$\\$ - \\$\\$\\$
Заменим пропуски на это значение

In [156]:
data['Price Range'] = data['Price Range'].fillna('$$ - $$$')

### 1.4. Признак Cuisine Style

In [157]:
data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')
#Заполняем пропуски значением Other
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: "['Cuisine_NOINFO']" if isinstance(x, float) else x)

In [158]:
data.info()

Пропуски устранены.

# 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [159]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

### 2.1. Признак City.

Добавим признак Rest_count_by_city - число ресторанов в городе, в котором расположен ресторан.

In [160]:
rest_count_by_city = dict(data['City'].value_counts())
data['rest_count_by_city'] = data.City.apply(lambda x: rest_count_by_city[x])

In [161]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
temp_df3 = pd.get_dummies(data['City'])
data = pd.concat([data,temp_df3],axis=1)

In [162]:
data.head(5)

In [163]:
data.sample(5)

### 2.2. Признак "Price Range".

In [164]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [165]:
# Ваша обработка 'Price Range'
data['Price_Rank'] = data['Price Range'].apply(lambda x: 1 if x == '$' else 2 if x == '$$ - $$$' else 3)

In [166]:
data['Price_Rank']

### 2.3. Признак Cuisine Style

Преобразуем значения в признаке Cuisine Style в двумерный список. Это позволит в дальнейшем удобно извлекать из него данные.

In [167]:
#cuisines - сет с перечнем всех встречающихся кухонь
cuisines = set()
#cus_list - итоговый список, который заменит значения в Cuisine Style.
cus_list = []
count = 0 
for current in data['Cuisine Style']:
    if pd.isna(current): continue
    ls = current.replace("[","").replace("]","").replace("'","")
    ls = ls.split(',')
    curr_list = []
    for cus in ls:
        #print(cus)
        if cus[0:1] == " ": 
            cuisines.add(cus[1:])
            curr_list.append(cus[1:])
        else: 
            cuisines.add(cus)
            curr_list.append(cus)
    
    cus_list.append(curr_list)

data['Cuisine Style'] = pd.Series(cus_list)


Разложим Топ10 встречающихся типов кухонь на dummies признаки.

In [168]:
#uniq_vals - словарь подсчета встречающихся значений для типов кухни 

uniq_vals = {}
for cus in cuisines:
    uniq_vals.update({cus: 0})
      
for row in data['Cuisine Style']:
    if isinstance(row, float): continue
    for cus in row:        
        uniq_vals.update({cus: uniq_vals[cus]+1})

In [169]:
#temp_df - временный DataFrame для раскладывания на dummies признаки
temp_df = pd.DataFrame.from_dict(uniq_vals,orient='index')
temp_df.columns = ['vals']
#Топ10 встречающихся кухонь
cuisine_top10 = temp_df['vals'].sort_values(ascending=False)[:10].index
#Остальные кухни
cuisine_other = list(set(temp_df.vals.index) - set(cuisine_top10))

#Раскладываем кухни на dummies - признаки
s = data['Cuisine Style']
temp_df2 = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)
#Для невошедших в топ10 создаем признак Cuisine_Other, означающий, что есть вхождения других типов кухонь
temp_df2['Cuisine_Other'] = temp_df2.apply(lambda x: 1 if x[cuisine_other].any() else 0, axis =1)

#добавим в топ10 еще тип Cuisine_Other, чтоб включить все эти типы в рабочий датафрейм
cuisine_top10 = list(cuisine_top10)
cuisine_top10.append('Cuisine_Other')

data = pd.concat([data,temp_df2[cuisine_top10]],axis=1)

### 2.4. Признак Reviews
Представлен ввиде строки, содержащей список с отзывами и датами. Преобразуем строку в реальный список. Извелчем два отдельных столбца: даты последнего и предпоследного отзывов

In [170]:
#Преоборазуем значения признака в список с помощью функции string_to_list
data['Reviews'] = data['Reviews'].apply(string_to_list)

Добавим признаки Reviews_last_date и Reviews_prelast_date типа DateTime - даты последнего и предпоследнего обзоров.

In [171]:
data['Reviews_last_date'] = data['Reviews'].apply(lambda x: pd.to_datetime(x[1][0]))
data['Reviews_prelast_date'] = data['Reviews'].apply(lambda x: 'NaT' if len(x[1]) < 2 else pd.to_datetime(x[1][1]))

Добавим признак дельты дней между последним и предспоследним отзывом

In [172]:
data['Reviews_delta'] = (data['Reviews_last_date'] - data['Reviews_prelast_date']).dt.days
#Заполним пропуски медианным значением
median = data['Reviews_delta'].median()
data['Reviews_delta'] = data['Reviews_delta'].fillna(median)

Добавим признак Last_Rev_Delta - Разница в днях относительно последним отзыва во всем датасете

In [173]:
data["Last_Rev_Delta"] = (data.Reviews_last_date.max() - data['Reviews_last_date'] ).dt.days
data["Last_Rev_Delta"] = data["Last_Rev_Delta"].fillna(data.Last_Rev_Delta.median())

Определим тональность отзывов. В результате получим два признака tone_polar - полярность от -1 до 1, и tone_subj - субъективная оценка текста от 0 до 1 (1 - очень субъективный текст)

In [174]:
final_scores = get_text_score(data['Reviews'])
data['tone_polar'] = pd.Series(final_scores[:,:,0].flatten())

### 2.5. Признак Ranking
Разделим Ranking на количество ресторанов в городе. В большом городе больше ресторанов и рейтинг соответственно может быть больше. 

In [175]:
data['Rank_per_City'] = data['Ranking'] / data['rest_count_by_city']

In [176]:
data.info()

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [177]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [178]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [179]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [180]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


Посмотрим на распределение признака Rank_per_City

In [181]:
for x in (data['City'].value_counts())[0:10].index:
    data['Rank_per_City'][data['City'] == x].hist(bins=100)
plt.show()

Вот и нормальное распредление. Rank_per_City - это признак, полученный делением Ranking на rest_count_by_city (количество ресторанов в городе). 

### Посмотрим распределение целевой переменной

In [182]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [183]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [184]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [185]:
plt.rcParams['figure.figsize'] = (20,20)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Кроме пары Cuisine_style_isNAN-Cuisine_NOINFO отсутствуют признаки с высокой корреляцией.

In [186]:
data[['Cuisine_Style_isNAN','Cuisine_NOINFO']].corr()


Коэффициент корреляции между 'Cuisine_Style_isNAN' и'Cuisine_NOINFO', равный 1, позволяет исключить один из признаков. Пусть это будет 'Cuisine_Style_isNAN'.

In [187]:
data = data.drop(['Cuisine_Style_isNAN'],axis=1)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [188]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [189]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['URL_TA','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    
    #Number of Reviews
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    # Заполняем пропуск по отзывам медианным значением отзывом для города, в котором расположен ресторан
    cities_name = df_output['City'].unique()
    #Получаем словарь индекс - город, значение - среднее количество отзывов по городу.
    num_rev_per_city = dict()
    for city in cities_name:
        num_rev_per_city[city] = df_output[df_output.City == city]['Number of Reviews'].median()
    df_output['Number of Reviews'] = df_output.apply(lambda row: num_rev_per_city[row.City] if np.isnan(row['Number of Reviews']) else row['Number of Reviews'], axis = 1)
    
    #Reviews
    #Заменим NaN на [[],[]]. Все остальные пустые значения в признаке Reviews обозначаются именно так. Позже их будет проще обработать в таком формате.
    df_output['Reviews'] = df_output['Reviews'].fillna("[[],[]]")
    
    #Price Range
    df_output['Price_Range_isNAN'] = pd.isna(df_output['Price Range']).astype('uint8')
    df_output['Price Range'] = df_output['Price Range'].fillna('$$ - $$$')
    
    #Cuisine Style
    #Заполняем пропуски значением Other
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: "['Cuisine_NOINFO']" if isinstance(x, float) else x)
    
    
    # ################### 3. Encoding ############################################################## 
    #rest_count_by_city - число ресторанов в городе, в котором расположен ресторан.
    rest_count_by_city = dict(df_output['City'].value_counts())
    df_output['rest_count_by_city'] = df_output.City.apply(lambda x: rest_count_by_city[x])
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    
    #Price Range
    df_output['Price_Rank'] = df_output['Price Range'].apply(lambda x: 1 if x == '$' else 2 if x == '$$ - $$$' else 3)
    
    #Cuisine Style
    #Преобразуем значения в признаке Cuisine Style в двумерный список. Это позволит в дальнейшем удобно извлекать из него данные.
    #cuisines - сет с перечнем всех встречающихся кухонь
    cuisines = set()
    #cus_list - итоговый список, который заменит значения в Cuisine Style.
    cus_list = []
    count = 0 
    for current in df_output['Cuisine Style']:
        if pd.isna(current): continue
        ls = current.replace("[","").replace("]","").replace("'","")
        ls = ls.split(',')
        curr_list = []
        for cus in ls:
            #print(cus)
            if cus[0:1] == " ": 
                cuisines.add(cus[1:])
                curr_list.append(cus[1:])
            else: 
                cuisines.add(cus)
                curr_list.append(cus)    
        cus_list.append(curr_list)
    df_output['Cuisine Style'] = pd.Series(cus_list)
    #uniq_vals - словарь подсчета встречающихся значений для типов кухни 
    uniq_vals = {}
    for cus in cuisines:
        uniq_vals.update({cus: 0})      
    for row in df_output['Cuisine Style']:
        if isinstance(row, float): continue
        for cus in row:        
            uniq_vals.update({cus: uniq_vals[cus]+1})
    #temp_df - временный DataFrame для раскладывания на dummies признаки
    temp_df = pd.DataFrame.from_dict(uniq_vals,orient='index')
    temp_df.columns = ['vals']
    #Топ10 встречающихся кухонь
    cuisine_top10 = temp_df['vals'].sort_values(ascending=False)[:10].index
    #Остальные кухни
    cuisine_other = list(set(temp_df.vals.index) - set(cuisine_top10))
    #Раскладываем кухни на dummies - признаки
    s = df_output['Cuisine Style']
    temp_df2 = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)
    #Для невошедших в топ10 создаем признак Cuisine_Other, означающий, что есть вхождения других типов кухонь
    temp_df2['Cuisine_Other'] = temp_df2.apply(lambda x: 1 if x[cuisine_other].any() else 0, axis =1)
    #добавим в топ10 еще тип Cuisine_Other, чтоб включить все эти типы в рабочий датафрейм
    cuisine_top10 = list(cuisine_top10)
    cuisine_top10.append('Cuisine_Other')
    df_output = pd.concat([df_output,temp_df2[cuisine_top10]],axis=1)
    
    #Reviews
    #Преоборазуем значения признака в список с помощью функции string_to_list
    df_output['Reviews'] = df_output['Reviews'].apply(string_to_list)
    



    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    
    
   
    
    #Reviews_delta - дельта дней между последним и предспоследним отзывом
    df_output['Reviews_last_date'] = df_output['Reviews'].apply(lambda x: pd.to_datetime(x[1][0]))
    df_output['Reviews_prelast_date'] = df_output['Reviews'].apply(lambda x: 'NaT' if len(x[1]) < 2 else pd.to_datetime(x[1][1]))
    df_output['Reviews_delta'] = (df_output['Reviews_last_date'] - df_output['Reviews_prelast_date']).dt.days
    #Заполним пропуски медианным значением
    median = df_output['Reviews_delta'].median()
    df_output['Reviews_delta'] = df_output['Reviews_delta'].fillna(median)
    
    #Last_Rev_Delta - Разница в днях относительно последним отзыва во всем датасете
    df_output["Last_Rev_Delta"] = (df_output.Reviews_last_date.max() - df_output['Reviews_last_date'] ).dt.days
    df_output["Last_Rev_Delta"] = df_output["Last_Rev_Delta"].fillna(df_output.Last_Rev_Delta.median())
    
    #tone_polar - суммарная полярность отзывов
    #final_scores = get_text_score(df_output['Reviews'])
    df_output['tone_polar'] = pd.Series(final_scores[:,:,0].flatten())
    
    #Rank_per_City - отношение 
    df_output['Rank_per_City'] = df_output['Ranking'] / df_output['rest_count_by_city']
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    #Удалим признаки с DateTime, а также более неинтересующие и rest_count_by_city
    df_output.drop(['Reviews_last_date','Reviews_prelast_date'], axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [190]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [191]:
df_preproc.info()

In [192]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [193]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [194]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [195]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [196]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [197]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [198]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [199]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [200]:
test_data.sample(10)

In [201]:
test_data = test_data.drop(['Rating'], axis=1)

In [202]:
sample_submission

In [203]:
predict_submission = model.predict(test_data)

In [204]:
predict_submission

In [205]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
